### IMPORT LIBRARIES FROM THE DIFFERENT FUNCTIONS

In [3]:
from utilities.logger import Logger
from utilities.utilities import *
import datetime
from tournaments.tournaments_wta import *
from matches.match_details import *
from matches.match import *
import time
import os
import functools
from typing import *
import pprint
import unidecode
import glob
import os

In [50]:
run_id = datetime.now().strftime('%Y%m%d_%H%M')
run_id


'20230910_1033'

In [51]:
logger = Logger(run_id)
logger.initialization()

## I -  Matches and matches_details mismatch
Here it's very specific as I missed some matches and matches details when retrieving existing tournaments
That shoud have been the first action here. I need to loop other the existings tournaments and correct missing data
 - cleaning empty files with no lines for the existing folders
 - Do we have **matches** and **matches_details** ? try to retrieve each one if not the case here
 - rewrite csv if necesseray (matches and matches details) to avoid duplicates due to scrapping method function: **rewrite_file_if_duplicates**
 we have to do that with **matches** and **matches_details**
 - for each existing folder, retrieve the lines of **matches** for whicn **match_details** has not been retrieved, so typically we assume we retrieve full matches and then we've to complete **matches_details** if necessary
 - retrieve the missing tournaments according to the date of the day

In [52]:
def create_path(*args):
    return os.path.join(*args)

### I.1 Delete files with no data in matches and matches_details

In [53]:
def delete_orphan_files(logger: Logger) -> None:
    """
    the goal here is to delete files for which we fail
    to retrieve matches details and the csv still exist
    with only headers or no data
    Args
    -------
    None
    Outputs
    -------
    None
    """
    years = os.listdir(os.path.join(os.getcwd(), "prd-trd", "0_raw"))
    logger.log_info(f"correcting data for years {','.join(years)}")
    base_path = os.path.join(os.getcwd(), "prd-trd", "0_raw")
    logger.log_info("="*80)
    #iterating over the years
    for year in years:
        logger.log_info(f"scanning year {year}......")
        year_path = os.path.join(base_path, year)
        #iterating over the tournaments of this year
        for tourney in os.listdir(year_path):
            tourney_path = os.path.join(year_path, tourney)
            matches_path = create_path(tourney_path, "matches.csv")
            matches_details_path = create_path(tourney_path, "matches_details.csv")
            common_file = set(["matches.csv", "matches_details.csv"]).intersection(os.listdir(tourney_path))

            if len(common_file)>0:
                try:
                    pd_matches = pd.read_csv(matches_path, sep=";")
                    if len(pd_matches)<2:
                        logger.log_info(f"deleting  {matches_path}")
                        os.remove(matches_path)
                    else:
                        logger.log_info(f"healthy file {matches_path}")

                except Exception as e:
                    logger.log_info(f"Can't read the matches for tourney {tourney}")
                
                try:
                    pd_matches_details = pd.read_csv(matches_details_path, sep=";")
                    if len(pd_matches_details)<2:
                        logger.log_info(f"deleting  {matches_details_path}")
                        os.remove(matches_details_path)
                    else:
                        logger.log_info(f"healthy file {matches_details_path}")
                except Exception as e:
                    logger.log_info(f"Can't read the matches details for tourney {tourney}")                  
                    if os.path.exists(matches_details_path):
                        logger.log_info(f"deleting  {matches_details_path} file corrupted")
                        os.remove(matches_details_path)
                       
            else:
                logger.log_info(f"nothing to scan for tourney {tourney_path}")



### I.2 Functions to scan and check if in the existing folders we have missing data

In [54]:
# verifiy if the PRD-TRD folder exist for the given year, otherwise, create it (problem with 30)
def generate_tournament_matches(path_of_tourney):
    """
    based on the tournament url, generate csv
    with matches played with basic stats as scores
    and time
    Args
    --------------
    tourney_path : the path to the tournament csv in
    the folders
    Outputs
    ------------
    the file generate in the folder given in input
    """
    container = "prd-trd"
    level = "0_raw"
    #print(f"im in the function input: {path_of_tourney}")
    tourney_name = path_of_tourney.split("\\")[-1]
    tournament_path = os.path.join(path_of_tourney, f"{tourney_name}.csv")
    #print(f"im in the function tournament path name: {tournament_path}")
    #print(f" tourney name: {tourney_name}")
    tournament = pd.read_csv(tournament_path, sep=";")
    size = len(tournament)
    year = path_of_tourney.split("\\")[-2]
    print(f"year: {year}")

    #get the list of the tourney based on tournaments_for_year
    for index, tourney in tournament.iterrows():
        print(f" donnees de tournoi brut: {len(tournament)}")
        print(f" donnees de tournoi deduplicated: {len(tournament.drop_duplicates())}")
        if index>=0:
            logger.log_info(f"processing tourney {index}/{size} {tourney['title']} for year {year}")
            data_tournament_values = retrieve_matches_html_robust(tourney['scores_link'])
            data_tournament = data_tournament_values[1]
            elapsed_time = data_tournament_values[0]
            logger.log_info(f"executing retrieving tournament {tourney['title']} matches  took {elapsed_time: .1f}s")
            #store the data
            os.makedirs(path_of_tourney, exist_ok=True)
            logger.log_info(f"tourney data will be stored here {path_of_tourney}")                                
            
            
            print(f"tourney link: {tourney['scores_link']}")
            data_tournament["scores_link"] = tourney['scores_link']
            if len(data_tournament)>2:
                #taking the matches of the tournament
                logger.log_info(f"data tournaments data will be stored here {path_of_tourney}")
                data_tournament.to_csv(os.path.join(path_of_tourney, "matches.csv"), sep=";", index=False)

def generate_tournament_matches_details(tourney_path):
    """
    based on the matches csv, generate csv
    with matches details 
    Args
    --------------
    tourney_path : the path to the csv folder of
    the tournament
    Outputs
    ------------
    the file generate in the folder given in input
    """
    # une autre boucle
    data_tournament_details = []
    tourney_time = 0
    tourney_name = tourney_path.split("\\")[-1]
    year = tourney_path.split("\\")[-2]
    mach_path = os.path.join(tourney_path, "matches.csv")
    data_tournament = pd.read_csv(mach_path, sep=";")
    print(f" donnees de match brut: {len(data_tournament)}")
    print(f" donnees de match deduplicated: {len(data_tournament.drop_duplicates())}")
    for index_match, match in data_tournament.drop_duplicates().iterrows():
        try:
            match_details_value = retrieving_match_data(match["url_match_stats"])
            match_details = match_details_value[1]
            elapsed_time = match_details_value[0]
            tourney_time += round(elapsed_time, 1)
            logger.log_info(f" elapsed time: {elapsed_time: .1f}s year: {year} tourney: {tourney_name} index: {index_match}/{len(data_tournament.drop_duplicates())}, stage: {match['stage']}")
        except Exception as e:
            logger.log_info(f"error treating line {index_match}/{len(data_tournament.drop_duplicates())}, {tourney_name}, url {match['url_match_stats']}")
        data_tournament_details.append(match_details)
        time.sleep(5)
    data_tournament_details_safe = [elt for elt in data_tournament_details if isinstance(elt, dict)]
    data_tournament_skipped = [elt for elt in data_tournament_details if not isinstance(elt, dict)]
    logger.log_info(f"retrieve all matches from {tourney_name} in {tourney_time: .1f}s")
    logger.log_info(f"matchs skipped for tourney {tourney_name}:{len(data_tournament_skipped)}")
    data_tournament_details_pd = pd.DataFrame(data_tournament_details_safe)
    #write to csv
    if len(data_tournament_details_pd)>2:
        #taking the matches of the tournament
        logger.log_info(f"data tournaments data will be stored here {tourney_path}")
        data_tournament_details_pd.to_csv(create_path(tourney_path,"matches_details.csv"), sep=";", index=False)
    time.sleep(60)

def case_missing_files_in_tourney_folder(logger):
    """
    for each folder based on the missing files
    configuration generate matches.csv
    matches_details.csv or both
    Args
    --------------
    None
    Outputs
    ------------
    matches, matches details, nothing or one
    of them
    """
    years = os.listdir(os.path.join(os.getcwd(), "prd-trd", "0_raw"))
    logger.log_info(f"correcting data for years {','.join(years)}")
    base_path = os.path.join(os.getcwd(), "prd-trd", "0_raw")
    #iterating over the years
    for year in years:
        logger.log_info(f"scanning year {year}......")
        year_path = os.path.join(base_path, year)
        #iterating over the tournaments of this year
        for tourney in os.listdir(year_path):
            tourney_path = os.path.join(year_path, tourney)
            logger.log_info(f"tourney path {tourney_path} of year {year}")
            logger.log_info(f"scanning tourney {tourney} of year {year}")
            potential_missing_csvs = ["matches.csv", "matches_details.csv"]
            csv_in_tourney_folder = os.listdir(os.path.join(year_path, tourney))
            if len(set(potential_missing_csvs).difference(csv_in_tourney_folder))==0:
                logger.log_info(f"no missing files for tourney {tourney} of year {year}")
            else:
                missing_files = list(set(potential_missing_csvs).difference(csv_in_tourney_folder))
                logger.log_info(f"missing files in  tourney: {','.join(missing_files)}")
                match missing_files:
                    #if we have only the tournament in the folder
                    case ["matches.csv", "matches_details.csv"]:
                        #we need to recompute both matches and matches_details
                        #first matches.csv based on tournament
                        print(f"tourney_path: {tourney_path}")
                        
                        generate_tournament_matches(tourney_path)
                        #second regenerate the matches_details
                        if "matches.csv" in os.listdir(tourney_path):
                            generate_tournament_matches_details(tourney_path)
                    
                    case ["matches_details.csv", "matches.csv"]:
                        #we need to recompute both matches and matches_details
                        #first matches.csv based on tournament
                        print(f"tourney_path: {tourney_path}")
                        
                        generate_tournament_matches(tourney_path)
                        #second regenerate the matches_details
                        if "matches.csv" in os.listdir(tourney_path):
                            generate_tournament_matches_details(tourney_path)
                    
                    case ["matches_details.csv"]:
                        #only generate matches details
                        if "matches.csv" in os.listdir(tourney_path):
                            generate_tournament_matches_details(tourney_path)
            logger.log_info("\n")    

### I.3 Deduplicate existing folders of tournaments

In [55]:
def create_path(*args):
    return os.path.join(*args)


def remove_duplicates_in_tourney_folders(logger):
    """
    for each folder based on the missing files
    configuration generate matches.csv
    matches_details.csv or both
    Args
    --------------
    None
    Outputs
    ------------
    matches, matches details, nothing or one
    of them
    """
    years = os.listdir(os.path.join(os.getcwd(), "prd-trd", "0_raw"))
    logger.log_info(f"correcting data for years {','.join(years)}")
    base_path = os.path.join(os.getcwd(), "prd-trd", "0_raw")
    #iterating over the years
    for year in years:
        logger.log_info(f"scanning year {year}...... \n")
        year_path = os.path.join(base_path, year)
        #iterating over the tournaments of this year
        for tourney in os.listdir(year_path):
            tourney_path = os.path.join(year_path, tourney)
            for file in os.listdir(tourney_path):
                file_path = os.path.join(tourney_path, file)
                logger.log_info(f"scanning: {file_path}....")
                pd_file = pd.read_csv(file_path, sep=";")
                pd_file_dedup = pd.read_csv(file_path, sep=";").drop_duplicates()
                logger.log_info(f"rows in file: {len(pd_file)}")
                logger.log_info(f"rows in file dedup: {len(pd_file_dedup)}")
                if len(pd_file)!=len(pd_file_dedup):
                    logger.log_info(f"duplicates in: {file_path}, must rewrite...")
                    pd_file_dedup.to_csv(file_path, sep=";", index=False)

                    


### I.4 Managing missing match_details for which we've match infos

In [81]:

def retrieving_missing_data(tourney_path: str, data_tournament: pd.DataFrame, is_trad_folder) -> None:
    """
    return the missing matches details for the
    given matches
    Args
        - tourney_path : tourney folder path
        - data_tournament : path to the data
        - is_trad_folder : verify that the folder is normal
                           with no existing extended data
    Returns
        - None (we complete existing csv with the matches csv)
    Hints
        - None
    """
    data_tournament_details = []
    tourney_time = 0
    tourney_name = tourney_path.split("\\")[-1]
    logger.log_info(f"tourney_name: {tourney_name}")
    year = tourney_path.split("\\")[-2]
    logger.log_info(f"toutney_path: {tourney_path}")
    logger.log_info(f"year: {year}")
    logger.log_info(f"type data tournament: {type(data_tournament)}")
    logger.log_info(f"len data tournament: {len(data_tournament)}")
    
    print(f" donnees de match brut: {len(data_tournament)}")
    print(f" donnees de match deduplicated: {len(data_tournament.drop_duplicates())}")
    for index_match, match in data_tournament.drop_duplicates().iterrows():
        try:
            match_details_value = retrieving_match_data(match["url_match_stats"])
            match_details = match_details_value[1]
            elapsed_time = match_details_value[0]
            tourney_time += round(elapsed_time, 1)
            logger.log_info(f" elapsed time: {elapsed_time: .1f}s year: {year} tourney: {tourney_name} index: {index_match}/{len(data_tournament.drop_duplicates())}, stage: {match['stage']}")
        except Exception as e:
            logger.log_info(f"error treating line {index_match}/{len(data_tournament.drop_duplicates())}, {tourney_name}, url {match['url_match_stats']}")
        data_tournament_details.append(match_details)
        time.sleep(5)
    data_tournament_details_safe = [elt for elt in data_tournament_details if isinstance(elt, dict)]
    data_tournament_skipped = [elt for elt in data_tournament_details if not isinstance(elt, dict)]
    logger.log_info(f"retrieve all matches from {tourney_name} in {tourney_time: .1f}s")
    logger.log_info(f"matchs skipped for tourney {tourney_name}:{len(data_tournament_skipped)}")
    data_tournament_details_pd = pd.DataFrame(data_tournament_details_safe)
    
    #taking the matches of the tournament
    logger.log_info(f"adding {len(data_tournament_details_pd)} matches details to {tourney_path}")
    #reading existing csv
    if is_trad_folder:
        existing_matches_details = pd.read_csv(os.path.join(tourney_path, "matches_details.csv"), sep=";")
    else:
        existing_matches_details = pd.read_csv(os.path.join(tourney_path, "matches_details_extended.csv"), sep=";")

    logger.log_info(f"len existing match details for  {tourney_path}:{len(existing_matches_details)}")
    matches_details_corrected = pd.concat([existing_matches_details, data_tournament_details_pd], ignore_index=True, axis=0)
    logger.log_info(f"len corrected match details for  {tourney_path}:{len(matches_details_corrected)}")
    matches_details_corrected.to_csv(os.path.join(tourney_path,"matches_details_extended.csv"), sep=";", index=False)
    print("*"*80)
    time.sleep(60)


def complete_missing_match_details(logger):
    """
    for each folder based on the missing files
    configuration generate matches.csv
    matches_details.csv or both
    Args
    --------------
    None
    
    Outputs
    ------------
    matches, matches details, nothing or one
    of them
    """
    years = os.listdir(os.path.join(os.getcwd(), "prd-trd", "0_raw"))
    logger.log_info(f"correcting data for years {','.join(years)}")
    base_path = os.path.join(os.getcwd(), "prd-trd", "0_raw")
    #iterating over the years
    for year in years:
        logger.log_info(f"scanning year {year}...... \n")
        year_path = os.path.join(base_path, year)
        #iterating over the tournaments of this year
        for tourney in os.listdir(year_path):
            tourney_path = os.path.join(year_path, tourney)
            try:
                matches_path = os.path.join(tourney_path, "matches.csv")
                logger.log_info(f"path of matches: {matches_path}")
                matches = pd.read_csv(matches_path, sep=";")
                logger.log_info(f"len matches: {len(matches)} for tourney {tourney}")
            except Exception as e:
                logger.log_info(f"impossible to open: {matches_path}")
                matches = None
                
            try:
                matches_details_path = os.path.join(tourney_path, "matches_details.csv")
                logger.log_info(f"path of matches details: {matches_details_path}")
                matches_details = pd.read_csv(matches_details_path, sep=";")
                logger.log_info(f"len matches details: {len(matches)} for tourney {tourney}")
            except Exception as e:
                logger.log_info(f"impossible to open: {matches_details_path}")
                matches_details = None

            #matches to retrieve
            try:
                matches_left = matches[~matches['url_match_stats'].isin(matches_details['url_match_stats'])]
                # retrieve matches details for matches left
                logger.log_info(f"missing: {len(matches_left)} matches details for tourney {tourney}")
                logger.log_info(f"retrieving missing data for tourney {tourney}.....")
                if len(matches_left)>0:
                    logger.log_info(f"calling retrieving_missing_data function")
                    is_trad_folder = os.path.exists(os.path.join(tourney_path, "matches_details_extended.csv"))
                    logger.log_info(f"does the folder containing extended.csv ? {is_trad_folder}.....")
                    retrieving_missing_data(tourney_path, matches_left, is_trad_folder)
                else:
                    logger.log_info(f"healthy match details for tourney {tourney_path}.....")
            except Exception as e:
                #logger.log_info(f"impossible to reach matchs_details csv file for tourney {tourney}")
                logger.log_info(f"error {e}")
            
            logger.log_info(f"\n \n")


    


In [82]:
complete_missing_match_details(logger)

correcting data for years 2018,2019,2020,2021,2022,2023
scanning year 2018...... 

path of matches: c:\Users\beAtz\OneDrive\Bureau\WTA_PROJECT\prd-trd\0_raw\2018\2018_Dongfeng_Motor_Wuhan_Open-_Wuhan,_China\matches.csv
len matches: 79 for tourney 2018_Dongfeng_Motor_Wuhan_Open-_Wuhan,_China
path of matches details: c:\Users\beAtz\OneDrive\Bureau\WTA_PROJECT\prd-trd\0_raw\2018\2018_Dongfeng_Motor_Wuhan_Open-_Wuhan,_China\matches_details.csv
len matches details: 79 for tourney 2018_Dongfeng_Motor_Wuhan_Open-_Wuhan,_China
missing: 1 matches details for tourney 2018_Dongfeng_Motor_Wuhan_Open-_Wuhan,_China
retrieving missing data for tourney 2018_Dongfeng_Motor_Wuhan_Open-_Wuhan,_China.....
healthy match details for tourney c:\Users\beAtz\OneDrive\Bureau\WTA_PROJECT\prd-trd\0_raw\2018\2018_Dongfeng_Motor_Wuhan_Open-_Wuhan,_China.....

 

path of matches: c:\Users\beAtz\OneDrive\Bureau\WTA_PROJECT\prd-trd\0_raw\2018\2018_Kunming_Open\matches.csv
len matches: 43 for tourney 2018_Kunming_Open


In [71]:
class ApplyingDataQualityToStack:
    """
    The aim of this class is to apply
    a whole range of check on the existing
    folders to ensure we didnt miss any data
    and files retrieved remain useable.
    """
    def __init__(self, logger: Logger)-> None:
        self.logger = logger

    def managing_orphan_files(self) -> None:
        """
        refer to the docstring of orphan_files
        """
        self.logger.log_info("*"*70)
        self.logger.log_info("Deleting orphan files in tourney folders")
        delete_orphan_files(self.logger)
        self.logger.log_info("******* end of Managing orphan files in tourney folders********")
    
    def managing_case_missing_files_in_tourney_folder(self) -> None:
        """
        refer to case_missing_files_in_tourney_folder docstring
        """
        self.logger.log_info("*"*70)
        self.logger.log_info("Managing missing files in tourney folders")
        case_missing_files_in_tourney_folder(self.logger)
        self.logger.log_info("******* end of Managing missing files in tourney folders********")
    
    def managing_remove_duplicates_in_tourney_folders(self) -> None:
        """
        refer to remove_duplicates_in_tourney_folders docstring
        """
        self.logger.log_info("*"*70)
        self.logger.log_info("Remove duplicates if necessary")
        remove_duplicates_in_tourney_folders(self.logger)
        self.logger.log_info("******* end of Remove duplicates if necessary********")
        
    def managing_complete_missing_match_details(self):
        """
        refer to the documentation of 
        retrieving_missing_data
        """
        self.logger.log_info("*"*70)
        self.logger.log_info("Completing missing matches_details....")
        complete_missing_match_details(self.logger)
        self.logger.log_info("******* end of complete_missing_match_details********")



In [72]:
data_quality_process = ApplyingDataQualityToStack(logger)

#delete files with no data
#data_quality_process.managing_orphan_files()

#case_missing_files_in_tourney_folder()
#data_quality_process.managing_case_missing_files_in_tourney_folder()

#remove_duplicates_in_tourney_folders()
#data_quality_process.managing_remove_duplicates_in_tourney_folders()

#retrieve missing matches
data_quality_process.managing_complete_missing_match_details()

**********************************************************************
Completing missing matches_details....
correcting data for years 2018,2019,2020,2021,2022,2023
scanning year 2018...... 

path of matches: c:\Users\beAtz\OneDrive\Bureau\WTA_PROJECT\prd-trd\0_raw\2018\2018_Dongfeng_Motor_Wuhan_Open-_Wuhan,_China\matches.csv
len matches: 79 for tourney 2018_Dongfeng_Motor_Wuhan_Open-_Wuhan,_China
path of matches details: c:\Users\beAtz\OneDrive\Bureau\WTA_PROJECT\prd-trd\0_raw\2018\2018_Dongfeng_Motor_Wuhan_Open-_Wuhan,_China\matches_details.csv
len matches details: 79 for tourney 2018_Dongfeng_Motor_Wuhan_Open-_Wuhan,_China
missing: 1 matches details for tourney 2018_Dongfeng_Motor_Wuhan_Open-_Wuhan,_China
retrieving missing data for tourney 2018_Dongfeng_Motor_Wuhan_Open-_Wuhan,_China.....
healthy match details for tourney c:\Users\beAtz\OneDrive\Bureau\WTA_PROJECT\prd-trd\0_raw\2018\2018_Dongfeng_Motor_Wuhan_Open-_Wuhan,_China.....

 

path of matches: c:\Users\beAtz\OneDrive\Bure

## Completing and reorganizing existing_data

### I - Tournaments

In [4]:
import re
from datetime import datetime

def parsing_tournament_date(date: str) -> tuple:
    """
    Take the date of the tournament and parse it to return:
    - Start date of the tournament
    - End date of the tournament 
    
    Args
    --------
    date : str
        The date to parse, e.g., ' Jul 3   Jul 16, 2023 '
        
    Returns
    -------
    tuple
        (date_debut, date_fin) : (03/07/2023, 16/07/2023)

    """
    
    # Normalize spaces to single spaces and trim the string
    normalized_date = ' '.join(date.strip().split())

    # Extract month-day pairs using regex
    dates = re.findall(r"\b\w{3} \d+", normalized_date)
    if len(dates) != 2:
        raise ValueError("Invalid date format!")
    
    # Extract the year from the normalized date string
    year = re.search(r"\d{4}", normalized_date)
    if not year:
        raise ValueError("Year not found in the date!")
    year = year.group(0)

    # Append the year to each date fragment
    dates_with_year = [f"{elt} {year}" for elt in dates]

    # Convert the date fragments to datetime format
    date_debut = datetime.strptime(dates_with_year[0], "%b %d %Y")
    date_fin = datetime.strptime(dates_with_year[1], "%b %d %Y")

    # Convert each datetime object to the desired string format
    """
    date_debut_str = date_debut.strftime("%d/%m/%Y")
    date_fin_str = date_fin.strftime("%d/%m/%Y")
    """
    
    return date_debut, date_fin



### I.2 Looping on each folder to create tournament.csv with updated informations

In [10]:
years = os.listdir(os.path.join(os.getcwd(),"prd-trd", "0_raw"))
base_path = os.path.join(os.getcwd(),"prd-trd", "0_raw")
years
for year in years:
    year_path = os.path.join(base_path, year)
    tourneys_for_year = os.listdir(year_path)
    for tourney in tourneys_for_year:
        try:
            tourney_folder = os.path.join(year_path, tourney)
            tournament = pd.read_csv(os.path.join(tourney_folder, f"{tourney}.csv"), sep=";") 
            date_parsing = tournament["dates"].apply(parsing_tournament_date)
            tournament["date_debut"] = date_parsing.apply(lambda x:x[0])
            tournament["date_fin"] = date_parsing.apply(lambda x:x[1])
            tourney_folder_quality = tourney_folder.replace("0_raw", "1_data_quality")
            #create folder
            os.makedirs(tourney_folder_quality, exist_ok=True)
            tourney_to_write = os.path.join(f"{tourney_folder_quality}", f"{tourney}.csv")
            #save new file
            if not os.path.exists(tourney_to_write):
                tournament.to_csv(tourney_to_write, sep=";",index=False)
            else:
                print("file already exist")

        except Exception as e:
            print(f"error when updating tournament: {e}")
            

file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already exist
file already